In [ ]:
# Modified from: https://github.com/britt-allen/ecfr_xml_parsing/blob/master/ecfr_parsing_notebook_with_beautiful_soup.ipynb
# https://towardsdatascience.com/scraping-multiple-urls-with-python-tutorial-2b74432d085f

In [ ]:
import warnings; warnings.filterwarnings("ignore")
from bs4 import BeautifulSoup as bs
from tqdm.notebook import tqdm_notebook
from random import randint
from time import sleep
import pandas as pd
import numpy as np
import requests
import time

In [ ]:
ncbi_link = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=gene&id="

In [ ]:
#genelist_link = "https://www.genenames.org/cgi-bin/download/custom?col=gd_app_sym&col=md_eg_id&status=Approved&status=Entry%20Withdrawn&hgnc_dbtag=on&limit=100&order_by=gd_app_sym_sort&format=text&submit=submit"
genelist_link = "https://www.genenames.org/cgi-bin/download/custom?col=gd_app_sym&col=md_eg_id&status=Approved&status=Entry%20Withdrawn&hgnc_dbtag=on&order_by=gd_app_sym_sort&format=text&submit=submit"

In [ ]:
df_genelist = pd.read_csv(genelist_link, header=0, sep="\t", error_bad_lines=False)
df_genelist

In [ ]:
genelist = df_genelist['NCBI Gene ID(supplied by NCBI)'].fillna(0).astype(int).tolist()
genelist = [x for x in genelist if str(x) != 'nan']
genelist = [x for x in genelist if str(x) != '0']
print(genelist)

In [ ]:
bs_data = []

for i in tqdm_notebook(genelist, desc = 'Progress: '):
    try:
        url_link = str(ncbi_link + str(i)); r = requests.get(url_link)
        soup = bs(r.content,'xml')
        name_bs = soup.find("Name").text
        description_bs = soup.find("Description").text
        species_bs = soup.find("ScientificName").text
        aliases_bs = soup.find("OtherAliases").text
        summary_bs = soup.find("Summary").text
        
        #print(i, '\t\t\t', name_bs, '\t\t\t', species_bs, '\t\t\t', description_bs)

        bs_data.append({'UID': i,
                        'ScientificName': species_bs,
                        'Description': description_bs,
                        'Name': name_bs,
                        'Summary': summary_bs})

    except AttributeError:
        print("Gene Number: " + str(i) + " Error!!!")
        
print("Done!!!")

In [ ]:
df_bs = pd.DataFrame(data=bs_data, columns=['UID', 'ScientificName', 'Name','Description', 'Summary'])

In [ ]:
df_bs.to_csv("output.csv", index=False, sep ='\t')